# Dimensionality Reduction Analysis

## Enviroment Setup

In [ ]:
%conda install -c conda-forge numpy pandas scikit-learn matplotlib seaborn -y

In [25]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import KMeans
import time

## Load Data

In [39]:
data = np.load('data/features.npz')
movieIds = data['movieId']
features = data['features']
movies = pd.concat([pd.read_csv('data/train_complete.csv'), pd.read_csv('data/test_complete.csv')], ignore_index=True)

print(f"Loaded features: {features.shape}")
print(f"Loaded movies: {len(movies)}")

Loaded features: (9337, 3345)
Loaded movies: 9337


## Standarize Features

In [40]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

n_components = 32

## PCA

In [41]:
start = time.time()
pca = PCA(n_components=n_components, random_state=42)
features_pca = pca.fit_transform(features_scaled)
pca_time = time.time() - start

print(f"Time: {pca_time:.2f}s")
print(f"Shape: {features_pca.shape}")
print(f"Explained Variance Ratio: {pca.explained_variance_ratio_.sum():.4f}")

Time: 1.56s
Shape: (9337, 32)
Explained Variance Ratio: 0.2585


## SVD

In [42]:
start = time.time()
svd = TruncatedSVD(n_components=n_components, random_state=42)
features_svd = svd.fit_transform(features_scaled)
svd_time = time.time() - start

print(f"Time: {svd_time:.2f}s")
print(f"Shape: {features_svd.shape}")
print(f"Explained Variance Ratio: {svd.explained_variance_ratio_.sum():.4f}")

Time: 1.18s
Shape: (9337, 32)
Explained Variance Ratio: 0.2582


## LDA

In [43]:
genres_series = movies.set_index('movieId')['genres'].reindex(movieIds)
genres_available = genres_series.notna()

genres_split = genres_series[genres_available].str.split('|')
primary_genre = genres_split.str[0]

unique_genres = primary_genre.value_counts()
top_genres = unique_genres[unique_genres >= 20].index.tolist()

genre_mask = primary_genre.isin(top_genres)

features_for_lda = features_scaled[genres_available][genre_mask]
labels_for_lda = primary_genre[genre_mask]

n_classes = len(labels_for_lda.unique())
n_components_lda = min(n_classes - 1, features_for_lda.shape[1], n_components)

start = time.time()
lda = LDA(n_components=n_components_lda)
features_lda = lda.fit_transform(features_for_lda, labels_for_lda)
lda_time = time.time() - start

print(f"Time: {lda_time:.2f}s")
print(f"Shape: {features_lda.shape}")
print(f"Explained variance: {lda.explained_variance_ratio_.sum():.4f}")
print(f"Movies used: {len(features_lda)}/{len(movieIds)}")

Time: 7.18s
Shape: (9321, 15)
Explained variance: 1.0000
Movies used: 9321/9337


## Evaluate with Clustering Metrics

In [44]:
methods = {'PCA': features_pca, 'SVD': features_svd, 'LDA': features_lda}

results = []

for method_name, features_reduced in methods.items():
    kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
    labels = kmeans.fit_predict(features_reduced)
    
    silhouette = silhouette_score(features_reduced, labels)
    davies_bouldin = davies_bouldin_score(features_reduced, labels)
    calinski_harabasz = calinski_harabasz_score(features_reduced, labels)
    
    results.append({
        'Method': method_name,
        'Silhouette': silhouette,
        'Davies-Bouldin': davies_bouldin,
        'Calinski-Harabasz': calinski_harabasz
    })

results_df = pd.DataFrame(results)
display(results_df)

,Method,Silhouette,Davies-Bouldin,Calinski-Harabasz
0,PCA,0.032429,3.149387,324.952332
1,SVD,0.030411,3.221409,325.429321
2,LDA,0.084182,2.005097,417.959045


## Save Results

In [45]:
np.savez_compressed('data/features_pca.npz', movieId=movieIds, features=features_pca)
np.savez_compressed('data/features_svd.npz', movieId=movieIds, features=features_svd)
np.savez_compressed('data/features_lda.npz', movieId=movieIds[genres_available][genre_mask], features=features_lda)

print("Reduced features sets saved.")

Reduced features sets saved.
